In [1]:
import pandas as pd

In [2]:
train_data_path = "../data/processed/train_data.csv"
test_data_path = "../data/processed/test_data.csv"

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [3]:
test_data

,AC,PMID,Title,Abstract,if_contain_keyterm,Terms,Text_combined
0,Q2G2U4,17827301,New insights into the WalK/WalR (YycG/YycF) es...,The highly conserved WalK/WalR (also known as ...,0,autolysis,New insights into the WalK/WalR (YycG/YycF) es...
1,P00520,20072125,Targeting Bcr-Abl by combining allosteric with...,In an effort to find new pharmacological modal...,0,autophosphorylation,Targeting Bcr-Abl by combining allosteric with...
2,Q8RXD3,15998807,The AIP2 E3 ligase acts as a novel negative re...,The phytohormone abscisic acid (ABA) mediates ...,0,autoubiquitination,The AIP2 E3 ligase acts as a novel negative re...
3,B0FLN1,18281398,Isolation and characterization of an autoinduc...,The opportunistic human pathogen Acinetobacter...,0,autoinduction,Isolation and characterization of an autoinduc...
4,P00519,16543148,Organization of the SH3-SH2 unit in active and...,The tyrosine kinase c-Abl is inactivated by in...,0,autoinhibition,Organization of the SH3-SH2 unit in active and...
5,P9WEN4,31117659,Distinct Autocatalytic α- N-Methylating Precur...,Backbone N-methylations impart several favorab...,0,autocatalysis,Distinct Autocatalytic - N-Methylating Precur...
6,Q9Y4W6,37917749,Autoregulatory control of mitochondrial glutat...,Mitochondria must maintain adequate amounts of...,0,autoregulation,Autoregulatory control of mitochondrial glutat...
7,P06213,12707268,Structural and biochemical evidence for an aut...,Tyrosine 984 in the juxtamembrane region of th...,0,"autoinhibition, autophosphorylation",Structural and biochemical evidence for an aut...
8,Q06486,7665585,Role of COOH-terminal phosphorylation in the r...,Casein kinase I delta is a member of the casei...,0,"autoinhibition, autophosphorylation",Role of COOH-terminal phosphorylation in the r...
9,Q99986,21543316,NMR solution structure of human vaccinia-relat...,Vaccinia-related kinase 1 (VRK1) is one of the...,0,"autocatalysis, autophosphorylation",NMR solution structure of human vaccinia-relat...


**Check for multi-labled terms**

In [4]:
test_data[test_data['Terms'].str.contains(',', na=False)]

,AC,PMID,Title,Abstract,if_contain_keyterm,Terms,Text_combined
7,P06213,12707268,Structural and biochemical evidence for an aut...,Tyrosine 984 in the juxtamembrane region of th...,0,"autoinhibition, autophosphorylation",Structural and biochemical evidence for an aut...
8,Q06486,7665585,Role of COOH-terminal phosphorylation in the r...,Casein kinase I delta is a member of the casei...,0,"autoinhibition, autophosphorylation",Role of COOH-terminal phosphorylation in the r...
9,Q99986,21543316,NMR solution structure of human vaccinia-relat...,Vaccinia-related kinase 1 (VRK1) is one of the...,0,"autocatalysis, autophosphorylation",NMR solution structure of human vaccinia-relat...
13,Q8IYU2,38332367,Structural mechanisms of autoinhibition and su...,Ubiquitin ligases (E3s) are pivotal specificit...,1,"autoinhibition, autoubiquitination",Structural mechanisms of autoinhibition and su...
14,O64517,21209078,Calcium-dependent activation and autolysis of ...,Metacaspases (MCPs) are members of a new famil...,1,"autocatalysis, autolysis",Calcium-dependent activation and autolysis of ...


**Check if there is data leakage**

In [5]:
common_pmids = set(train_data['PMID']).intersection(set(test_data['PMID']))
common_pmids

set()

**Check Term normalization**

In [6]:
terms_series = train_data['Terms'].str.split(',').explode()
terms_series = terms_series.str.strip()
term_counts = terms_series.value_counts()
term_counts

Terms
autophosphorylation    8450
autoregulation         2360
autoinhibition         1830
autocatalysis          1790
autoubiquitination     1430
autoinduction           830
autolysis               670
autoactivation          220
autofeedback            130
autokinase               80
Name: count, dtype: int64

**Check terms in text**

In [7]:
def check_terms_in_text(row):
    if pd.isna(row['Terms']) or pd.isna(row['Text_combined']):
        return 0
    terms = [term.strip() for term in row['Terms'].split(',')]
    return 1 if any(term.lower() in row['Text_combined'].lower() for term in terms) else 0

In [8]:
train_data['if_contain_keyterm'] = train_data.apply(check_terms_in_text, axis=1)
train_data['if_contain_keyterm'].value_counts()

if_contain_keyterm
0    43890
1     8640
Name: count, dtype: int64

In [9]:
test_data['if_contain_keyterm'] = test_data.apply(check_terms_in_text, axis=1)
test_data[test_data['Terms'].notna()]['if_contain_keyterm'].value_counts()

if_contain_keyterm
0    10
1     5
Name: count, dtype: int64